In [4]:
def solution(start, s):
    if not s:
        return
    
    min_val = min(s)
    min_idx = s.index(min_val)

    res[start+min_idx] = min_val
    print(''.join(res))
    solution(start+min_idx, s[min_idx+1:])

    solution(start, s[:min_idx])


s = input()
res = [''] * len(s)
solution(0, s)

A
C
OC
ZOC


In [11]:
s = input()
boom = input()
while True:
    flag = False
    if boom in s:
        s = s.replace(boom, '')
        flag = True
    
    if not flag:
        break

if s:
    print(s)
else:
    print('FRULA')

mirkovniz


In [13]:
s = input()
boom = input()

stack = []
ex_len = len(boom)
for i in range(len(s)):
    stack.append(s[i])
    if ''.join(stack[-ex_len:]) == boom:
        for _ in range(ex_len):
            stack.pop()

if stack:
    print(''.join(stack))
else:
    print('FRULA')

mirkovniz


In [17]:
sum([i for i in range(1, 20)])

190

In [25]:
n, m = map(int, input().split())
board = [list(map(int, input().split())) for _ in range(n)]
ch = [[0]*n for _ in range(n)]
res = 0
dx = [-1,0,1,0]
dy = [0,1,0,-1]

for _ in range(m):
    x, y = map(int, input().split())
    x, y = x-1, y-1
    res += board[x][y]
    time = 0

    while time != 3:
        time += 1
        tmp = 0
        for i in range(4):
            xx = x + dx[i]
            yy = y + dy[i]
            if 0<=xx<n and 0<=yy<n and ch[xx][yy] == 0:
                if board[xx][yy] > tmp:
                    tmp = board[xx][yy]
                    t_x = xx
                    t_y = yy
        
        x, y = t_x, t_y
        ch[x][y] = 1
        res += board[x][y]

print(res)

0 2
0 3
1 3
2 2
2 3
3 3


In [27]:
res

633

In [22]:
board

[[20, 26, 185, 80], [100, 29, 25, 80], [20, 20, 88, 99], [15, 32, 44, 50]]

In [23]:
ch

[[1, 1, 0, 0], [1, 1, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]